# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 31 2023 11:11AM,256286,10,MSP218058,"Methapharm, Inc.",Released
1,Jan 31 2023 11:11AM,256286,10,MSP218038,Methapharm-G,Released
2,Jan 31 2023 11:11AM,256286,10,MSP218054,Methapharm-G,Released
3,Jan 31 2023 11:09AM,256285,19,2022102799,"GUSA Granules USA, Inc.",Released
4,Jan 31 2023 11:09AM,256283,10,0086342314,ISDIN Corporation,Released
5,Jan 31 2023 11:09AM,256283,10,0086342315,ISDIN Corporation,Released
6,Jan 31 2023 11:09AM,256283,10,0086342324,ISDIN Corporation,Released
7,Jan 31 2023 11:09AM,256283,10,0086342325,ISDIN Corporation,Released
8,Jan 31 2023 11:09AM,256283,10,0086342326,ISDIN Corporation,Released
9,Jan 31 2023 11:09AM,256283,10,0086342327,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,256282,Released,1
33,256283,Released,22
34,256284,Released,1
35,256285,Released,1
36,256286,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
256282,NaN,1.0
256283,NaN,22.0
256284,NaN,1.0
256285,NaN,1.0
256286,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256149,0.0,2.0
256198,0.0,2.0
256218,0.0,1.0
256236,6.0,23.0
256240,24.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256149,0,2
256198,0,2
256218,0,1
256236,6,23
256240,24,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256149,0,2
1,256198,0,2
2,256218,0,1
3,256236,6,23
4,256240,24,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256149,,2
1,256198,,2
2,256218,,1
3,256236,6,23
4,256240,24,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 31 2023 11:11AM,256286,10,"Methapharm, Inc."
1,Jan 31 2023 11:11AM,256286,10,Methapharm-G
3,Jan 31 2023 11:09AM,256285,19,"GUSA Granules USA, Inc."
4,Jan 31 2023 11:09AM,256283,10,ISDIN Corporation
26,Jan 31 2023 11:08AM,256284,10,ISDIN Corporation
27,Jan 31 2023 11:06AM,256282,10,"Senseonics, Incorporated"
28,Jan 31 2023 10:54AM,256280,10,"Senseonics, Incorporated"
29,Jan 31 2023 10:52AM,256278,16,Sartorius Bioprocess Solutions
31,Jan 31 2023 10:43AM,256276,18,ECI Pharmaceuticals LLC
32,Jan 31 2023 10:30AM,256273,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 31 2023 11:11AM,256286,10,"Methapharm, Inc.",,3
1,Jan 31 2023 11:11AM,256286,10,Methapharm-G,,3
2,Jan 31 2023 11:09AM,256285,19,"GUSA Granules USA, Inc.",,1
3,Jan 31 2023 11:09AM,256283,10,ISDIN Corporation,,22
4,Jan 31 2023 11:08AM,256284,10,ISDIN Corporation,,1
5,Jan 31 2023 11:06AM,256282,10,"Senseonics, Incorporated",,1
6,Jan 31 2023 10:54AM,256280,10,"Senseonics, Incorporated",,1
7,Jan 31 2023 10:52AM,256278,16,Sartorius Bioprocess Solutions,,2
8,Jan 31 2023 10:43AM,256276,18,ECI Pharmaceuticals LLC,,1
9,Jan 31 2023 10:30AM,256273,10,Bio-PRF,,14


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 31 2023 11:11AM,256286,10,"Methapharm, Inc.",3,
1,Jan 31 2023 11:11AM,256286,10,Methapharm-G,3,
2,Jan 31 2023 11:09AM,256285,19,"GUSA Granules USA, Inc.",1,
3,Jan 31 2023 11:09AM,256283,10,ISDIN Corporation,22,
4,Jan 31 2023 11:08AM,256284,10,ISDIN Corporation,1,
5,Jan 31 2023 11:06AM,256282,10,"Senseonics, Incorporated",1,
6,Jan 31 2023 10:54AM,256280,10,"Senseonics, Incorporated",1,
7,Jan 31 2023 10:52AM,256278,16,Sartorius Bioprocess Solutions,2,
8,Jan 31 2023 10:43AM,256276,18,ECI Pharmaceuticals LLC,1,
9,Jan 31 2023 10:30AM,256273,10,Bio-PRF,14,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 31 2023 11:11AM,256286,10,"Methapharm, Inc.",3,
1,Jan 31 2023 11:11AM,256286,10,Methapharm-G,3,
2,Jan 31 2023 11:09AM,256285,19,"GUSA Granules USA, Inc.",1,
3,Jan 31 2023 11:09AM,256283,10,ISDIN Corporation,22,
4,Jan 31 2023 11:08AM,256284,10,ISDIN Corporation,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 31 2023 11:11AM,256286,10,"Methapharm, Inc.",3.0,NaN
1,Jan 31 2023 11:11AM,256286,10,Methapharm-G,3.0,NaN
2,Jan 31 2023 11:09AM,256285,19,"GUSA Granules USA, Inc.",1.0,NaN
3,Jan 31 2023 11:09AM,256283,10,ISDIN Corporation,22.0,NaN
4,Jan 31 2023 11:08AM,256284,10,ISDIN Corporation,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 31 2023 11:11AM,256286,10,"Methapharm, Inc.",3.0,0.0
1,Jan 31 2023 11:11AM,256286,10,Methapharm-G,3.0,0.0
2,Jan 31 2023 11:09AM,256285,19,"GUSA Granules USA, Inc.",1.0,0.0
3,Jan 31 2023 11:09AM,256283,10,ISDIN Corporation,22.0,0.0
4,Jan 31 2023 11:08AM,256284,10,ISDIN Corporation,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5125190,152.0,9.0
15,768776,100.0,40.0
16,256278,2.0,0.0
18,256276,1.0,0.0
19,1537583,33.0,6.0
21,256218,1.0,0.0
22,256245,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5125190,152.0,9.0
1,15,768776,100.0,40.0
2,16,256278,2.0,0.0
3,18,256276,1.0,0.0
4,19,1537583,33.0,6.0
5,21,256218,1.0,0.0
6,22,256245,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,152.0,9.0
1,15,100.0,40.0
2,16,2.0,0.0
3,18,1.0,0.0
4,19,33.0,6.0
5,21,1.0,0.0
6,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,152.0
1,15,Released,100.0
2,16,Released,2.0
3,18,Released,1.0
4,19,Released,33.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,21,22
Status,,,,,,,
Executing,9.0,40.0,0.0,0.0,6.0,0.0,0.0
Released,152.0,100.0,2.0,1.0,33.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,21,22
0,Executing,9.0,40.0,0.0,0.0,6.0,0.0,0.0
1,Released,152.0,100.0,2.0,1.0,33.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,21,22
0,Executing,9.0,40.0,0.0,0.0,6.0,0.0,0.0
1,Released,152.0,100.0,2.0,1.0,33.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()